In [1]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup
import re

### Lấy category id

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en;q=0.9,en-US;q=0.8',
    'Referer': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789',
    'x-guest-token': 'f1kXJZDbVQ8N0PFzH3E92yKUeg4Wa6wY',
    'Connection': 'keep-alive',
}

params = {
    'limit': '40',
    'include': 'advertisement',
    'aggregations': '2',
    'trackity_id': '563cc097-0278-3210-d9eb-2b08ef7b99a0',
    'category': '1789',
    'page': '1',
}

In [5]:
categories_id = []
repo = requests.get("https://tiki.vn/", headers=headers, params=params)
soup = BeautifulSoup(repo.text, 'html.parser')

soup = soup.find('div', class_='styles__StyledListItem-sc-w7gnxl-0 cjqkgR')
a = soup.find_all('a')
for i in a:
    match = re.search(r'/c(\d+)', i['href'])
    categories_id.append(match.group(1))

categories_id

['8322',
 '1883',
 '1789',
 '2549',
 '1815',
 '1882',
 '1520',
 '8594',
 '931',
 '4384',
 '1975',
 '915',
 '17166',
 '1846',
 '1686',
 '4221',
 '1703',
 '1801',
 '27498',
 '44792',
 '8371',
 '6000',
 '11312',
 '976',
 '27616',
 '15078']

### Lấy id của sản phẩm

In [6]:
product_id = []
for j in categories_id:
    params['category'] = j
    for i in range(1, 2):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)
        if response.status_code == 200:
            # print("requests complete")
            for record in response.json().get('data'):
                product_id.append({'id': record.get('id')})
print(len(product_id))

1415


In [7]:
df = pd.DataFrame(product_id)
df = set(df['id'])
df = pd.DataFrame(df)
df.rename(columns={0: 'id'}, inplace=True)
print(df)

             id
0     273317890
1     193151000
2     170721330
3     185991219
4     270483515
...         ...
1330   32923633
1331  192135155
1332  127787001
1333   74547194
1334     745469

[1335 rows x 1 columns]


### Lấy comment trên từng sản phẩm

In [ ]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en;q=0.9,en-US;q=0.8',
    'Referer': 'https://tiki.vn/ke-toan-via-he-thuc-hanh-bao-cao-tai-chinh-can-ban-tu-quay-ban-nuoc-chanh-p262977989.html?',
    'x-guest-token': 'pUYHub5GIMswKcLtmv6gzl9yDFo8Cd7E',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = {
    'product_id': '',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'limit': '10',
    'include': 'comments'
}

def comment_parser(json):
    d = dict()
    d['id'] = json.get(id)
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    d['created_at'] = json.get('created_at')
    d['customer_name'] = json.get('created_by').get('full_name')
    d['purchased_at'] = json.get('created_by').get('purchased_at')
    return d

p_ids = df.id.to_list()[:10]
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    params['product_id'] = pid
    print('Crawl comment for product {}'.format(pid))
    for i in range(1):
        params['page'] = i
        yes = 1
        while(yes):
            try:
                response = requests.get('https://tiki.vn/api/v2/reviews?', headers=headers, params=params)
                if response.status_code == 200:
                    for comment in response.json().get('data'):
                        result.append(comment_parser(comment))
                    print('Crawl comment page {} success!!!'.format(i))
                yes = 0
            except:
                print('Crawl data {} fail !!!'.format(pid))
        
df_comment = pd.DataFrame(result)
df_comment.to_csv('comments_data_ncds.csv', index=False)

In [13]:
response = requests.get('https://tiki.vn/api/v2/reviews?', headers=headers, params=params)
print(response.status_code)

200


In [7]:
df_comment

,id,title,content,thank_count,customer_id,rating,created_at,customer_name,purchased_at
0,None,Cực kì hài lòng,"Là nhà đầu tư mê *** húc lên, tuy vậy không ph...",11,19456693,5,1666681120,Sarah Mỹ Liên,1665063360
1,None,Cực kì hài lòng,"Tiki giao hàng rất nhanh, mình đặt giao hàng t...",6,5479340,5,1664176384,Lê Đức Trung,1664088217
2,None,Rất không hài lòng,Vô cùng khó chịu với cách làm bookcover của ha...,0,24716981,1,1666614985,Nguyễn Anh Tú,1666543185
3,None,Bình thường,Cuốn sách thiên về tư duy của các nhà giao dịc...,3,21500215,3,1687147042,Bui Van Minh,1683278179
4,None,Rất không hài lòng,Mình đọc cuốn sách này khá ức chế. Sau hơn 800...,7,6144330,1,1664291782,Trần Thanh Tùng,1663044771
...,...,...,...,...,...,...,...,...,...
73,None,Hài lòng,Sản phẩm đúng mô tả. Phần hộp đựng cũng bằng s...,0,11239170,4,1676444108,Thanh Pham,1676029246
74,None,Hài lòng,HÀI LÒNG,1,16811399,4,1670245107,Nam Le,1668686702
75,None,Hài lòng,,0,6546190,4,1672588459,Khánh Trần,1661569267
76,None,Cực kì hài lòng,,0,12004510,5,1658133919,Nguyễn Nam Thanh,1657198218


### Lấy thông tin sản phẩm

In [ ]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html?src=category-page-1789&2hi=0',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = (
    ('platform', 'web'),
    ('spid', 187266106)
)

def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['sku'] = json.get('sku')
    d['short_description'] = json.get('short_description')
    d['price'] = json.get('price')
    d['list_price'] = json.get('list_price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    d['review_count'] = json.get('review_count')
    d['all_time_quantity_sold'] = json.get('all_time_quantity_sold')
    d['inventory_status'] = json.get('inventory_status')
    d['stock_item_qty'] = json.get('stock_item').get('qty')
    d['stock_item_max_sale_qty'] = json.get('stock_item').get('max_sale_qty')
    d['product_name'] = json.get('name')
    d['brand_id'] = json.get('brand').get('id')
    d['brand_name'] = json.get('brand').get('name')
    return d

p_ids = df.id.to_list()
print(p_ids)
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    print(pid)
    yes = 1
    while(yes):
        try:
            response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)
            print('Crawl data {} success !!!'.format(pid))
            result.append(parser_product(response.json()))
            yes = 0
        except:
            print('Crawl data {} fail !!!'.format(pid))
        
df_product = pd.DataFrame(result)
df_product.to_csv('crawled_data_ncds.csv', index=False)

In [38]:
print(result)

[{'id': 273258825, 'sku': '9616299554457', 'short_description': '\xa0...', 'price': 3290000, 'list_price': 3950000, 'discount': 660000, 'discount_rate': 17, 'review_count': 70, 'all_time_quantity_sold': 302, 'inventory_status': 'available', 'stock_item_qty': 1000, 'stock_item_max_sale_qty': 1000, 'product_name': 'Điện thoại Samsung Galaxy A05s (4GB/128GB) - Đã kích hoạt bảo hành điện tử - Hàng chính hãng', 'brand_id': 18802, 'brand_name': 'Samsung'}, {'id': 273951550, 'sku': '6075632127726', 'short_description': 'Thiết kế và Màn hìnhĐiện Thoại Samsung Galazy A15 LTE (8GB/128GB) thiết kế viền nổi độc đáo Key Island, mang lại trải nghiệm cầm nắm khác biệt, chắc chắn và dễ dàng.Màn hình Super AMOLED 90Hz cho t...', 'price': 3990000, 'list_price': 4990000, 'discount': 1000000, 'discount_rate': 20, 'review_count': 48, 'all_time_quantity_sold': 269, 'inventory_status': 'available', 'stock_item_qty': 1000, 'stock_item_max_sale_qty': 1000, 'product_name': 'Điện Thoại Samsung Galaxy A15 LTE (8G